In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# bus_groupを8つ読み込む
bus_groups = {}
for i in range(8):
    bus_groups[f"bus_group{i+1}"] = pd.read_excel(
        "../data/rust_data_clean.xlsx", sheet_name=i
    )

bus_groups["bus_group1"]

In [ ]:
def create_period_column(df, year_col, month_col, new_col) -> None:
    """指定された年と月の列から新しいPeriod列を作成する。

    Args:
        df (DataFrame): 処理するデータフレーム。
        year_col (str): 年のデータを含む列の名前。
        month_col (str): 月のデータを含む列の名前。
        new_col (str): 新しく作成するPeriod列の名前。
    """
    condition = df[year_col].notna() & df[month_col].notna()
    df.loc[condition, new_col] = pd.to_datetime(
        df.loc[condition, year_col].astype(int).astype(str)
        + "-"
        + df.loc[condition, month_col].astype(int).astype(str),
        format="%Y-%m",
    ).dt.to_period("M")
    df.drop(columns=[year_col, month_col], inplace=True)

In [ ]:
bus_groups_replaceinfo = pd.DataFrame()
for i in range(8):
    bus_replaceinfo = bus_groups[f"bus_group{i+1}"][0:10].T
    bus_replaceinfo["group"] = str(i + 1)
    bus_groups_replaceinfo = pd.concat(
        [bus_groups_replaceinfo, bus_replaceinfo], axis=0
    )

bus_groups_replaceinfo = bus_groups_replaceinfo.replace(0, np.nan)
bus_groups_replaceinfo.reset_index(inplace=True)
bus_groups_replaceinfo.rename(
    columns={
        "index": "bus_name",
        0: "month_purchased",
        1: "year_purchased",
        2: "month_1st_rep",
        3: "year_1st_rep",
        4: "odometer_1st",
        5: "month_2and_rep",
        6: "year_2and_rep",
        7: "odometer_2and",
        8: "month_data_begins",
        9: "year_data_begins",
    },
    inplace=True,
)

cols = ["year_purchased", "year_1st_rep", "year_2and_rep", "year_data_begins"]
for col in cols:
    bus_groups_replaceinfo[col] = bus_groups_replaceinfo[col].apply(
        lambda x: x if x == np.nan else 1900 + x
    )

create_period_column(
    bus_groups_replaceinfo, "year_purchased", "month_purchased", "purchase_date"
)
create_period_column(
    bus_groups_replaceinfo, "year_1st_rep", "month_1st_rep", "1st_rep_date"
)
create_period_column(
    bus_groups_replaceinfo, "year_2and_rep", "month_2and_rep", "2and_rep_date"
)
create_period_column(
    bus_groups_replaceinfo, "year_data_begins", "month_data_begins", "data_begins_date"
)

bus_groups_replaceinfo

In [ ]:
bus_groups_replaceinfo_summary = (
    bus_groups_replaceinfo[["group", "odometer_1st"]]
    .groupby("group")
    .agg(["max", "min", "mean", "std"])
    .round(0)
)
bus_groups_replaceinfo_summary

In [ ]:
def calculate_months_diff(date1: pd.Series, date2: pd.Series) -> pd.Series:
    result = np.where(
        date1.isna() | date2.isna(),
        np.nan,
        (date2.dt.year - date1.dt.year) * 12 + (date2.dt.month - date1.dt.month) + 1,
    )
    return pd.Series(result)

In [ ]:
bus_groups_replaceinfo["1st_rep_period"] = calculate_months_diff(
    bus_groups_replaceinfo["purchase_date"], bus_groups_replaceinfo["1st_rep_date"]
)
bus_groups_replaceinfo["2and_rep_period"] = calculate_months_diff(
    bus_groups_replaceinfo["1st_rep_date"], bus_groups_replaceinfo["2and_rep_date"]
)

bus_groups_replace_period = pd.DataFrame()
bus_groups_replace_1st_rep_period = (
    bus_groups_replaceinfo[["group", "1st_rep_period"]]
    .rename(columns={"1st_rep_period": "period"})
    .dropna()
)
bus_groups_replace_2and_rep_period = (
    bus_groups_replaceinfo[["group", "2and_rep_period"]]
    .rename(columns={"2and_rep_period": "period"})
    .dropna()
)

bus_groups_replace_period = pd.concat(
    [bus_groups_replace_1st_rep_period, bus_groups_replace_2and_rep_period], axis=0
)
bus_groups_replace_period_summary = (
    bus_groups_replace_period.groupby("group")
    .agg(["max", "min", "mean", "std"])
    .round(1)
)
bus_groups_replace_period_summary

In [ ]:
pd.merge(
    bus_groups_replaceinfo_summary,
    bus_groups_replace_period_summary,
    left_index=True,
    right_index=True,
)